In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

ModuleNotFoundError: No module named 'analystcommunity'

In [ ]:
query = """
SELECT
    s.identifier_value AS region_code,
    fs.order_close_date::DATE AS close_date,
    dp.card_id,
    dp.card_description AS product_name,
    nullif(SUM(fs.product_quantity_x_step_unit),0) AS cant,
    SUM((fs.product_price * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / cant as product_pricee,
    SUM((fs.product_price_discount * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / cant as product_price_discountt,
    1 - (product_price_discountt / product_pricee) AS dct,
    SUM((pb.min_price * fs.product_quantity_x_step_unit)) / cant as benchmark,
    1 - (SUM(cogs_p_day * fs.product_quantity_x_step_unit) / NULLIF(SUM(fs.gmv_pxq_local), 0)) AS margin,
    1 - (SUM(cogs_p_day * fs.product_quantity_x_step_unit) / NULLIF(SUM(fs.gmv_pxq_local - COALESCE(fsd.product_discount, 0)), 0)) AS net_margin,
    product_pricee/benchmark AS gpi,
    product_price_discountt/benchmark AS npi,
    cant*product_pricee AS gmv_usd,
    gmv_usd / SUM(gmv_usd) OVER (PARTITION BY s.identifier_value, fs.order_close_date::DATE) AS gmv_mix
    
FROM dpr_sales.fact_sales                                   fs
INNER JOIN dpr_shared.dim_customer                          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site                              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product                           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category                          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts                    fsd ON fs.order_item_id = fsd.order_item_id  
INNER JOIN dpr_shared.dim_stock_unit                        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit                        sup ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
INNER JOIN dpr_product_pricing.obt_benchmark_product_prices pb  ON pb.source_product_id = dp.source_id AND CONVERT_TIMEZONE ('UTC',s.timezone_code,fs.order_submitted_date)::DATE = pb.benchmark_date
INNER JOIN dpr_cross_business.fact_cross_business_insights  cbi ON cbi.dim_stock_unit = sup.stock_unit_id AND cbi.dim_date = fs.dim_close_date

WHERE 
    fs.gmv_enabled = TRUE
    AND cat.super_category = 'Fruver'
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND fs.order_close_date::DATE >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
    AND fs.order_close_date::DATE <  DATE_TRUNC('day', date(getdate()) + interval '1 day')
GROUP BY 1,2,3,4
"""
df = run_read_dwd_query(query)

In [22]:
today = (datetime.datetime.today()).strftime('%Y-%m-%d')

In [23]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [24]:
query = """
WITH 
gmv_month AS (
SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE_TRUNC('week', fs.order_submitted_date) AS week,
    dp.card_id,
    sum(fs.gmv_local) as gmv_week
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('week', date(getdate()) - interval '0 week')
GROUP BY 1,2,3
),

gmv_all AS (
SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE_TRUNC('week', fs.order_submitted_date) AS week,
    --dp.card_id,
    sum(fs.gmv_local) as gmv_all
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_product_pricing.obt_benchmark_product_prices pb ON pb.source_product_id = dp.source_id AND DATE(fs.order_submitted_date) = pb.benchmark_date

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('week', date(getdate()) - interval '0 week')
GROUP BY 1,2
)

SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    cat.parent_description as category,
    cat.description AS subcategory,
    dp.card_id,
    dp.card_description AS product_name,
    gm.gmv_week/ga.gmv_all AS gmv_mix_product,
    AVG(margin_mtd) AS margin,
    sum((fs.product_price * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / nullif(sum(fs.product_quantity_x_step_unit),0) as product_pricee,
    sum((fs.product_price_discount * fs.product_quantity_x_step_unit)+fs.product_tax_iva) / nullif(sum(fs.product_quantity_x_step_unit),0) as product_price_discountt,
    sum(fs.gmv_local) as gmv,
    AVG(pb.min_price) AS benchmark,
    sum(fs.gmv_local)/gm.gmv_week as gmv_mix,
    product_price_discountt/benchmark AS npi,
    round(1-(benchmark*0.95/product_pricee),4) AS dct
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id  
INNER JOIN dpr_product_pricing.obt_benchmark_product_prices pb ON pb.source_product_id = dp.source_id AND DATE(fs.order_submitted_date) = pb.benchmark_date
INNER JOIN gmv_month gm ON gm.card_id = dp.card_id AND gm.week = DATE_TRUNC('week', fs.order_submitted_date) AND gm.region_code = s.identifier_value
INNER JOIN gmv_all ga ON ga.week = DATE_TRUNC('week', fs.order_submitted_date) AND ga.region_code = s.identifier_value
INNER JOIN dpr_cross_business.fact_cross_business_insights cbi ON cbi.dim_stock_unit = pb.stock_unit_id AND cbi.dim_date = fs.dim_submitted_date

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('day', date(getdate()) - interval '0 day')
GROUP BY 1,2,3,4,5,6,gm.gmv_week,ga.gmv_all
HAVING gmv_mix_product > 0.005 AND npi > 0.98 AND npi < 1.5 AND dct < margin*1.1
"""
df = run_read_dwd_query(query)

In [25]:
df.loc[df.region_code.isin(['BOG','MDE'])]

,region_code,submit_date,category,subcategory,card_id,product_name,gmv_mix_product,margin,product_pricee,product_price_discountt,gmv,benchmark,gmv_mix,npi,dct
7,BOG,2023-10-16,Frutas & verduras,Frutas,-200028,Limón Tahití,0.0162,0.2729,5762.8900,5571.0773,396075.20260,5000.0000,0.1393,1.1142,0.1758
12,MDE,2023-10-16,Frutas & verduras,Verduras,280195,Plátano Verde,0.0065,0.2786,3010.0000,3010.0000,14527.90590,2900.0000,0.0597,1.0379,0.0848
16,MDE,2023-10-16,Frutas & verduras,Verduras,338426,Repollo Blanco,0.0054,0.1577,3420.0000,3420.0000,204870.18530,3000.0000,1.0000,1.1400,0.1667
36,MDE,2023-10-16,Frutas & verduras,Tubérculos,287486,Papa Capira,0.0214,0.4532,3574.3862,3480.0152,497045.64820,3132.3000,0.6188,1.1110,0.1675
39,BOG,2023-10-16,Frutas & verduras,Tubérculos,46983,Papa Blanca Sin Lavar,0.0091,0.5099,2074.9787,2008.6036,346721.60970,1913.2631,0.2161,1.0498,0.1241
43,MDE,2023-10-16,Frutas & verduras,Verduras,280198,Plátano Maduro,0.0070,0.2761,3520.0000,3520.0000,21506.99680,3300.0000,0.0820,1.0666,0.1094
46,BOG,2023-10-16,"Carne, pollo & pescados",Pollo fresco,19739,Pechuga con Hueso y Piel Sin Marinar Mediana P...,0.0120,0.1457,14222.2200,13655.2075,2097476.68720,13760.0000,1.0000,0.9923,0.0809
47,BOG,2023-10-16,Desechables,Contenedores,26182,Contenedor Sopa 16 Oz con Tapa Plástica Darnel,0.0071,0.1150,5776.7404,5650.0004,690562.58250,5700.0000,0.5540,0.9912,0.0627
51,BOG,2023-10-16,Lácteos & huevos,Huevos,136259,Amarre Huevos Rojos AA,0.0304,0.0550,101743.5900,99200.0000,5311987.76810,100800.0000,1.0000,0.9841,0.0589
57,MDE,2023-10-16,Desechables,Contenedores,426468,Contenedor 12 oz tapa plástica Darnel x 20 und,0.0095,0.1167,4620.7489,4538.6890,357963.98010,4600.0000,1.0000,0.9866,0.0543


In [26]:
df = df.loc[df.card_id.isin(frida_products.id_tarjeta.unique())].copy()

In [27]:
query = """
SELECT
  source_id AS customer_id,
  recency_days,
  last_order_city,
  business_type
FROM dpr_customer.obt_customer
WHERE recency_days <= 30
  AND (
    (business_type = 'Restaurante')
    OR
    (last_order_city = 'BAQ' AND business_type IN ('Restaurante', 'Comercio'))
  )
"""
df_customers = run_read_dwd_query(query)

In [28]:
query = """
SELECT 
s.identifier_value AS city,
dc.source_id AS customer_id,
COUNT(DISTINCT fs.order_id) AS orders,
CASE
    WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN SUM(fs.gmv_pxq_local)/4.75
    WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN SUM(fs.gmv_pxq_local)/3776
    WHEN s.identifier_value IN ('CMX') THEN SUM(fs.gmv_pxq_local)/19.65
    ELSE SUM(fs.gmv_pxq_local)
END AS gmv_usd,
gmv_usd/orders AS aov

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 

WHERE fs.gmv_enabled = TRUE
AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
AND fs.fb_order_status_id  IN (1,6,7,8)
AND DATE(fs.order_submitted_date) >= DATE_TRUNC('day', date(getdate()) - interval '30 day')
AND fs.is_deleted = FALSE
AND fs.dim_status = 1
AND dp.is_slot = 'false'
AND s.identifier_value IN ('BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP')
AND dc.source_id IN {clientes}

GROUP BY 1,2
""".format(clientes=tuple(df_customers.customer_id.unique()))

data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["gmv_usd", "orders","aov"]] = data_ventas_aux[["gmv_usd", "orders","aov"]].astype(float)

In [29]:
from sklearn.model_selection import train_test_split

def create_groups(offer):
    customers = offer.customer_id.unique()
    
    alpha = 0.01
    aux = 0
    
    while True:
        customer_control, customer_test = train_test_split(customers, test_size=0.50)
        
        avg_gmv_control = offer[offer.customer_id.isin(customer_control)]['aov'].mean()
        avg_gmv_test = offer[offer.customer_id.isin(customer_test)]['aov'].mean()
        
        # Check if both average GMV and average products meet the criteria
        if (
            abs(1 - (avg_gmv_control / avg_gmv_test)) <= alpha
        ):
            break
        
        aux += 1
        
        if aux == 1000:
            alpha += 0.01
            aux = 0
    print(alpha)
    return customer_control.tolist(), customer_test.tolist()

In [30]:
result_rows = []
for city in data_ventas_aux.city.unique():
    control, piloto = create_groups(data_ventas_aux.loc[data_ventas_aux.city == city])
    for customer_id in control:
        result_rows.append([city, customer_id, 'control'])
    for customer_id in piloto:
        result_rows.append([city, customer_id, 'piloto'])
result_df = pd.DataFrame(result_rows, columns=['city','customer_id', 'group'])

0.01
0.01
0.01
0.01
0.01
0.01
0.01
0.01


In [31]:
def segments(offer, cityy):
        import requests
        import json
        
        fecha = (datetime.datetime.today() + datetime.timedelta(days=(1))).strftime('%Y-%m-%d')
        global_segment_list = [] #//for procesing
        segment = {'name':f'EXTRA_INV_{cityy}_TOP_SKUS', 'customersIds': (offer.customer_id.unique()).tolist(),"expiresAt": fecha}
        global_segment_list.append(segment) 
        
        
        url = f"https://{cityy}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

        payload = json.dumps(global_segment_list)


        headers = {
        'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
        'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        
        sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
        ssgtt = pd.DataFrame(sgt, columns=['segment_id', 'name'])
        
        return ssgtt                

In [32]:
def csv_files(offer,segment):
    csv_file = {} 
    csv_file['offer_name'] = offer.offer_name
    csv_file['offer_description'] = None
    csv_file['discount'] = offer.dct
    csv_file['automatically_consider_offer'] = "true"
    csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')
    csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(1))).strftime('%d-%m-%Y')
    csv_file['max_uses_per_order'] = 0
    csv_file['max_uses_per_customer'] = 2
    csv_file['customer_segment_id'] = [segment.loc[segment.name == '_'.join(name.split('_')[:3]) + '_TOP_SKUS'].segment_id.unique()[0] for name in offer.offer_name]

    sku_ids = []
    for sku in offer.card_id:
        try:
            sku_id = frida_products.loc[frida_products.id_tarjeta == sku, "skus"].values[0]
        except IndexError:
            sku_id = None  # Assign a null value if IndexError occurs
        sku_ids.append(sku_id)
    
    csv_file['sku_id'] = sku_ids
    
    df = pd.DataFrame(csv_file)
    
    # Drop rows with null values in the 'sku_id' column
    df = df.dropna(subset=['sku_id'])
    return df

In [33]:
df.loc[df['dct'] > 15, 'dct'] = 15

In [34]:
df["dct"] = df.dct * 100
df["offer_name"] = (
        "EXTRA_INV_" +
        df["region_code"] + "_" +
        df["product_name"].str.replace(",", ".") + "_" + today + "_" +
        df["dct"].astype(str) + "%" + "_Pipe2"
        )

In [35]:
for city in df.region_code.unique():
    print(city)
    segment = segments(result_df.loc[(result_df.city == city) & (result_df.group == 'piloto')],city)
    
    city_csv = csv_files(df.loc[df.region_code == city],segment)
    city_csv = city_csv.drop_duplicates()

    # Concatenate the values in the 'customer_segment_id' column with "#"
    city_csv['customer_segment_id'] = city_csv.groupby('sku_id')['customer_segment_id'].transform(lambda x: '#'.join(x.astype(str)))

    # Drop duplicate rows
    city_csv = city_csv.drop_duplicates(subset=['sku_id'])

    # Reset index if needed
    city_csv = city_csv.reset_index(drop=True)

    city_csv = city_csv.loc[city_csv.discount > 0].copy()

    city_csv.to_csv(f"{city}_{today}_EXP_DCT_SKUS_PUSH_DAILY.csv", index = False)

CMX
SPO
BHZ
CWB
BOG
BAQ
MDE
VCP


In [36]:
result_df.to_csv(f"EXTRA_INV_ALL_PUSH_SKUS_{today}.csv")